In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import pandas as pd
import time, random
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service



In [42]:
# pip install selenium-wire
# # webdriver-manager
# # pip install 


In [43]:
PROXY_LIST = [
    "hppimpgq:fwqss0z8ktob@142.111.48.253:7030",
    "hppimpgq:fwqss0z8ktob@31.59.20.176:6754",
    "hppimpgq:fwqss0z8ktob@38.170.176.177:5572",
    "hppimpgq:fwqss0z8ktob@198.23.239.134:6540",
    "hppimpgq:fwqss0z8ktob@45.38.107.97:6014",
    "hppimpgq:fwqss0z8ktob@142.111.67.146:5611",
    "hppimpgq:fwqss0z8ktob@107.172.163.27:6543",
    "hppimpgq:fwqss0z8ktob@64.137.96.74:6641",
    "hppimpgq:fwqss0z8ktob@216.10.27.159:6837",
    "hppimpgq:fwqss0z8ktob@142.147.128.93:6593",
]

In [70]:
import os
import zipfile
import json
import textwrap
import random
import string
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# ---------------------------
# Utilities: build extension
# ---------------------------
def _random_name(n=6):
    return "".join(random.choices(string.ascii_lowercase, k=n))

def make_proxy_auth_extension(proxy_host: str, proxy_port: str, proxy_user: str, proxy_pass: str, scheme: str = "http"):
    """
    Create a Chrome extension (zip) that sets an upstream proxy and supplies auth credentials.
    Returns absolute path to the created .zip extension.
    """
    plugin_file = f"proxy_auth_{proxy_host}_{proxy_port}_{_random_name()}.zip"

    manifest_json = {
        "version": "1.0.0",
        "manifest_version": 2,
        "name": "Proxy Auth Extension",
        "permissions": [
            "proxy",
            "tabs",
            "unlimitedStorage",
            "storage",
            "<all_urls>",
            "webRequest",
            "webRequestBlocking"
        ],
        "background": {"scripts": ["background.js"]},
        "minimum_chrome_version":"22.0.0"
    }

    background_js = textwrap.dedent(f"""
    var config = {{
      mode: "fixed_servers",
      rules: {{
        singleProxy: {{ scheme: "{scheme}", host: "{proxy_host}", port: parseInt({proxy_port}) }},
        bypassList: ["localhost"]
      }}
    }};
    chrome.proxy.settings.set({{value: config, scope: "regular"}}, function(){{}});
    function callbackFn(details) {{
        return {{authCredentials: {{username: "{proxy_user}", password: "{proxy_pass}"}}}};
    }}
    chrome.webRequest.onAuthRequired.addListener(
        callbackFn,
        {{urls: ["<all_urls>"]}},
        ['blocking']
    );
    """)

    with zipfile.ZipFile(plugin_file, 'w') as zp:
        zp.writestr("manifest.json", json.dumps(manifest_json))
        zp.writestr("background.js", background_js)

    return os.path.abspath(plugin_file)

# ---------------------------
# Create driver function
# ---------------------------
def create_driver(proxy: str | None = None, headless: bool = True, allow_headless_with_ext: bool = False):
    """
    Create a Chrome webdriver.
    - proxy: None or "host:port" or "user:pass@host:port"
    - headless: if True, attempt headless; if extension required and headless not supported, will default to visible
    - allow_headless_with_ext: set to True to force headless even if extension may not load (not guaranteed)
    """
    opts = Options()
    # prefer the newer headless implementation
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--disable-extensions")  # we'll add required extensions explicitly
    opts.add_argument("--disable-gpu")

    extension_path = None
    # If proxy provided, determine if it has auth
    if proxy:
        if "@" in proxy:  # user:pass@host:port
            creds, hostport = proxy.rsplit("@", 1)
            if ":" not in creds or ":" not in hostport:
                raise ValueError("Auth proxy must be user:pass@host:port")
            user, pwd = creds.split(":", 1)
            host, port = hostport.split(":", 1)
            # Chrome historically doesn't support extensions in classic headless.
            # New headless may support; if not, switch to visible unless user allows.
            if headless and not allow_headless_with_ext:
                # warn and switch to visible because extensions often don't load in headless
                print("[warning] Auth-proxy requires extension; switching to visible Chrome (headless disabled). "
                      "Set allow_headless_with_ext=True if you want to attempt headless anyway.")
                headless = False
                # recreate options for visible mode
                opts = Options()
                opts.add_argument("--no-sandbox")
                opts.add_argument("--disable-dev-shm-usage")
                opts.add_argument("--disable-gpu")

            extension_path = make_proxy_auth_extension(host, port, user, pwd)
            opts.add_extension(extension_path)
        else:
            # no-auth proxy: use --proxy-server directly
            opts.add_argument(f"--proxy-server=http://{proxy}")

    # Create service and driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=opts)

    # set reasonable timeouts
    driver.set_page_load_timeout(30)
    driver.implicitly_wait(5)

    # return both driver and extension path so caller can cleanup
    return driver, extension_path

# def create_driver(proxy=None):
#     chrome_opts = Options()
#     chrome_opts.add_argument("--headless=new")
#     chrome_opts.add_argument("--no-sandbox")
#     chrome_opts.add_argument("--disable-dev-shm-usage")

#     if proxy:
#         chrome_opts.add_argument(f'--proxy-server=http://{proxy}')

#     driver = webdriver.Chrome(
#         service=Service(ChromeDriverManager().install()),
#         options=chrome_opts
#     )
#     return driver
# def _base_options(headless: bool):
#     opts = Options()
#     if headless:
#         opts.add_argument("--headless=new")   # hidden browser
#     # else visible (no headless flag)
#     opts.add_argument("--no-sandbox")
#     opts.add_argument("--disable-dev-shm-usage")
#     # optional: set a reasonable user agent if you want
#     # opts.add_argument("user-agent=MyScraperBot/1.0")
#     return opts
# def create_driver_headless(proxy: str | None = None):
#     """Return a headless Chrome WebDriver. proxy format: 'host:port' or 'user:pass@host:port' (see notes)."""
#     opts = _base_options(headless=True)
#     if proxy:
#         opts.add_argument(f"--proxy-server=http://{proxy}")
#     service = Service(ChromeDriverManager().install())
#     driver = webdriver.Chrome(service=service, options=opts)
#     return driver

# def create_driver_visible(proxy: str | None = None):
#     """Return a visible (non-headless) Chrome WebDriver."""
#     opts = _base_options(headless=False)
#     if proxy:
#         opts.add_argument(f"--proxy-server=http://{proxy}")
#     service = Service(ChromeDriverManager().install())
#     driver = webdriver.Chrome(service=service, options=opts)
#     return driver

In [77]:
# for proxy in PROXY_LIST:
#     browser = create_driver(proxy)
print(2)

2


In [75]:
# # browser = webdriver.Chrome()
browser, ext = create_driver("hppimpgq:fwqss0z8ktob@142.111.48.253:7030", headless=True, allow_headless_with_ext=True)

browser.get("https://httpbin.org/ip")
print(browser.find_element(By.TAG_NAME, "body").text)
# time.sleep(2)

{
  "origin": "105.69.81.66"
}


In [80]:
import undetected_chromedriver as uc

proxy = "hppimpgq:fwqss0z8ktob@142.111.48.253:7030"

options = uc.ChromeOptions()
options.add_argument("--headless=new")
options.add_argument(f'--proxy-server=http://{proxy.split("@")[1]}')

driver = uc.Chrome(options=options)
driver.get("https://httpbin.org/ip")
print(driver.page_source)


<html><head></head><body></body></html>


In [78]:
browser, ext = create_driver("142.111.48.253:7030", headless=True)
browser.get("https://httpbin.org/ip")
print(browser.find_element(By.TAG_NAME, "body").text)


In [47]:
# recherche = browser.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')

# time.sleep(2)
# recherche.send_keys("commercial kitchen equipment")
# confirmer = browser.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]')
# confirmer.click()
for n in range(2,11):
    product_divs = browser.find_elements(By.CSS_SELECTOR, 'div[role="listitem"]')
    for div in product_divs:
        a_tag = div.find_element(By.TAG_NAME, "a")
        href = a_tag.get_attribute("href")
        if href:
            print(href)


2


In [25]:
def get_element_text(driver, xpath):
    """
    Quickly returns the text of an element if found.
    If not found, returns an empty string immediately (no waiting or errors).
    """
    try:
        elements = driver.find_elements(By.XPATH, xpath)
        if elements:
            return elements[0].text.strip()
        else:
            return "Nan"
    except Exception:
        return "Nan"

In [26]:
def get_element_text_css(driver, css_selector):
    """
    Quickly return text for a CSS selector, or empty string if not found.
    Much faster than waiting for exceptions.
    """
    try:
        elements = driver.find_elements(By.CSS_SELECTOR, css_selector)
        return elements[0].text.strip() if elements else ""
    except Exception:
        return ""

In [31]:
from urllib.parse import urljoin
driver = browser
product_list = [] 

for n in range(2,3):
    product_urls = []
    product_divs = browser.find_elements(By.CSS_SELECTOR, 'div[role="listitem"]')
    for div in product_divs:
        try:
            a_tag = div.find_element(By.TAG_NAME, "a")
            href = a_tag.get_attribute("href")
            if href:
                product_urls.append(href)
        except:
            continue 
    browser.quit()       
    for proxy in PROXY_LIST:
        browser = create_driver(proxy)
        for link in product_urls:
            browser.get(link)
            time.sleep(0.5)
    
            product_info = {
                "name": get_element_text(driver, "//span[@id='productTitle']"),
                "price": get_element_text(driver, "//div[contains(@id, 'corePrice')]"),    
                "price_helper": get_element_text(driver, "//div[@id='corePriceDisplay_desktop_feature_div']"),
                "past month" : get_element_text(driver, "//div[@class='a-section a-spacing-micro social-proofing-faceout']"),
                "Brand": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Brand']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Brand')]]//td[2]"),
                "Recommended Uses For Product": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Recommended Uses For Product']/following-sibling::td[@class='a-size-base prodDetAttrValue']| //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Recommended Uses For Product')]]//td[2]" ),
                # "Mounting Type": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Mounting Type']/following-sibling::td[@class='a-size-base prodDetAttrValue']| //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Mounting Type')]]//td[2]" ),
                # "Finish Type": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Finish Type']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Finish Type')]]//td[2]" ),
                "Material": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Material']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Material')]]//td[2]" ),
                "Color": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Color']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Color')]]//td[2] | //span[@id='inline-twister-expanded-dimension-text-color_name']"),
                # "Number of Handles": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Number of Handles']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Number of Handles')]]//td[2]" ),
                # "Included Components": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Included Components']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Included Components')]]//td[2]" ),
                # "Hole Count": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Hole Count']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Hole Count')]]//td[2]" ),
                # "Special Feature": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Special Feature']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Special Feature')]]//td[2]" ),
                # "Style": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Style']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Style')]]//td[2] | //span[@id='inline-twister-expanded-dimension-text-style_name']"),
                # "Installation Type": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Installation Type']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Installation Type')]]//td[2]" ),
                # "Number of Pieces": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Number of Pieces']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Number of Pieces')]]//td[2]" ),
                # "Handle Material": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Handle Material']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Handle Material')]]//td[2]" ),
                # "Spout Height": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Spout Height']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Spout Height')]]//td[2]" ),
                # "Spout Reach": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Spout Reach']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Spout Reach')]]//td[2]" ),
                # "Item Weight": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Item Weight'][1]/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Item Weight')]]//td[2]" ),
                # "Series ID": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Series ID']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Series ID')]]//td[2]" ),
                # "Warranty Type": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Warranty Type']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Warranty Type')]]//td[2]" ),
                # "Handle Location": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Handle Location']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Handle Location')]]//td[2]" ),
                # "Speaker Cutout Diameter or Length": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Speaker Cutout Diameter or Length']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Speaker Cutout Diameter or Length')]]//td[2]" ),
                # "Spout Design": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Spout Design']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Spout Design')]]//td[2]" ),
                # "Manufacturer": get_element_text(driver, "//span[contains(@class, 'a-text-bold') and contains(., 'Manufacturer') and not(contains(., 'Discontinued'))]/following-sibling::*[1] | //div[@id='prodDetails']//th[normalize-space(.)='Manufacturer']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Manufacturer')]]//td[2]" ),            
                # "Product Dimensions": get_element_text(driver, "//div[@id='detailBulletsWrapper_feature_div']//span[contains(@class, 'a-text-bold') and contains(., 'Product Dimensions')]/following-sibling::span | //div[@id='prodDetails']//th[normalize-space(.)='Product Dimensions']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Product Dimensions')]]//td[2]" ),
                # "Part Number": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Part Number']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Part Number')]]//td[2]" ),
                # "Item model number": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Item model number']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Item model number')]]//td[2]" ),
                # "Is Discontinued By Manufacturer": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Is Discontinued By Manufacturer']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Is Discontinued By Manufacturer')]]//td[2]" ),
                # "Finish": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Finish']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Finish')]]//td[2]" ),
                # "Pattern": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Pattern']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Pattern')]]//td[2]" ),
                # "Shape": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Shape']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Shape')]]//td[2]" ),
                # "Item Package Quantity": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Item Package Quantity']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Item Package Quantity')]]//td[2]" ),
                # "Hose Length": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Hose Length']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Hose Length')]]//td[2]" ),
                # "Flush Type": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Flush Type']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Flush Type')]]//td[2]" ),
                # "Special Features": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Special Features']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Special Features')]]//td[2]" ),
                # "Batteries Included?": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Batteries Included?']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Batteries Included?')]]//td[2]" ),
                # "Batteries Required?": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Batteries Required?']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Batteries Required?')]]//td[2]" ),
                # "Warranty Description": get_element_text(driver, "//div[@id='prodDetails']//th[normalize-space(.)='Warranty Description']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'Warranty Description')]]//td[2]" ),
                "ASIN": get_element_text(driver, "//div[@id='detailBulletsWrapper_feature_div']//span[contains(@class, 'a-text-bold') and contains(., 'ASIN')]/following-sibling::span | //div[@id='prodDetails']//th[normalize-space(.)='ASIN']/following-sibling::td[@class='a-size-base prodDetAttrValue'] | //div[@id='productOverview_feature_div']//tr[.//span[contains(., 'ASIN')]]//td[2]" ),
                "rating": get_element_text(driver, "//*[@id='acrPopover']/span[1]/a/span"),
                "Reviews Count": get_element_text(driver, "//span[@id='acrCustomerReviewText']"),
                "5 star": get_element_text_css(driver, "#histogramTable li:nth-child(1) .a-text-right.aok-nowrap"),
                "4 star": get_element_text_css(driver, "#histogramTable li:nth-child(2) .a-text-right.aok-nowrap"),
                "3 star": get_element_text_css(driver, "#histogramTable li:nth-child(3) .a-text-right.aok-nowrap"),
                "2 star": get_element_text_css(driver, "#histogramTable li:nth-child(4) .a-text-right.aok-nowrap"),
                "1 star": get_element_text_css(driver, "#histogramTable li:nth-child(5) .a-text-right.aok-nowrap"),
                "Best Sellers Rank": get_element_text(driver, "//div[@id='detailBulletsWrapper_feature_div']//li[.//span[contains(@class, 'a-text-bold') and contains(., 'Best Sellers Rank')]]//span[@class='a-list-item'] | //div[@id='prodDetails']//th[normalize-space(.)='Best Sellers Rank']/following-sibling::td//span[@class='a-list-item']"),
                "Date First Available": get_element_text(driver, "//div[@id='detailBulletsWrapper_feature_div']//span[contains(@class, 'a-text-bold') and contains(., 'Date First Available')]/following-sibling::span | //div[@id='prodDetails']//th[normalize-space(.)='Date First Available']/following-sibling::td[@class='a-size-base prodDetAttrValue']"),
                "link": link
            }
    
            product_list.append(product_info)
            time.sleep(1)        
     
    browser.get(f"https://www.amazon.com/s?k=travel&page={n}") 
    time.sleep(0.7)

In [37]:
df = pd.DataFrame(product_list)
# print("product_list")

In [38]:
df.head(1000)

""


In [147]:
df.to_csv('products.csv', index=False)
print("Data saved to appartements.csv")
browser.quit()

Data saved to appartements.csv


In [11]:
ss.to_csv('test_see_columns.csv', index=False)

In [10]:
ss = df[["name","Brand","price","rating","Reviews Count","Recommended Uses For Product","Material","Style","Color","ASIN","5 star","4 star","3 star","2 star","1 star", "Finish Type", "Warranty Type", "Best Sellers Rank", "past month", "Date First Available","link"]]
# [df.columns.get_loc('Manufacturer'), df.columns.get_loc('link'), df.columns.get_loc('Product Dimensions')]]


In [35]:
print(2)

2


In [82]:
import requests

proxy = "user:pass@142.111.48.253:7030"
for proxy in PROXY_LIST:
    if not proxy.startswith("http"):
        proxy = "http://" + proxy

    proxies = {"http": proxy, "https": proxy}

    try:
        r = requests.get("https://httpbin.org/ip", proxies=proxies, timeout=10)
        print("OK", r.status_code, r.text)
    except Exception as e:
        print("Proxy test failed:", e)


OK 200 {
  "origin": "142.111.48.253"
}

OK 200 {
  "origin": "31.59.20.176"
}

OK 200 {
  "origin": "38.170.176.177"
}

OK 200 {
  "origin": "198.23.239.134"
}

OK 200 {
  "origin": "45.38.107.97"
}

OK 200 {
  "origin": "142.111.67.146"
}

OK 200 {
  "origin": "107.172.163.27"
}

OK 200 {
  "origin": "64.137.96.74"
}

OK 200 {
  "origin": "216.10.27.159"
}

OK 200 {
  "origin": "142.147.128.93"
}



In [ ]:
print(5)

In [ ]:
pip install selenium webdriver-manager proxy.py

In [94]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# Specify the path to ChromeDriver
chrome_driver_path = "/usr/local/bin/chromedriver"  # Adjust if needed
proxy = "username:password@proxy_host:proxy_port"

# Set up Chrome options
options = webdriver.ChromeOptions()
options.add_argument(f'--proxy-server=http://{proxy}')
# Optional: Specify Chrome binary location if not in default path
options.binary_location = "/usr/bin/google-chrome"  # Adjust if Chrome is elsewhere
# Optional: Run headless to avoid opening a browser window
options.add_argument('--headless')
options.add_argument('--no-sandbox')  # Required for WSL/Linux environments
options.add_argument('--disable-dev-shm-usage')  # Avoids issues in WSL

# Initialize ChromeDriver
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)

# Test the proxy
driver.get("https://api.ipify.org")
print(driver.page_source)  # Should print the proxy IP
driver.get("https://whatismyipaddress.com")

# Clean up
driver.quit()

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [102]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import os

# Proxy details (replace with your actual proxy)
proxy = "hppimpgq:fwqss0z8ktob@142.111.48.253:7030"

# Set up Chrome options
options = Options()
options.add_argument(f'--proxy-server=http://{proxy}')
options.add_argument('--headless=new')  # New headless mode for Chrome 141+
options.add_argument('--no-sandbox')  # Required for WSL
options.add_argument('--disable-dev-shm-usage')  # Avoid shared memory issues
options.add_argument('--disable-gpu')  # Disable GPU
options.add_argument('--disable-webrtc')  # Prevent WebRTC leaks
options.add_argument('--ignore-certificate-errors')  # Handle proxy SSL issues
options.binary_location = "/opt/google/chrome/google-chrome"  # Real binary path (not the wrapper script)

# Set up ChromeDriver path
chrome_driver_path = "/usr/bin/chromedriver"

# Verify paths
print(f"Chrome binary exists: {os.path.exists('/opt/google/chrome/google-chrome')}")
print(f"ChromeDriver exists: {os.path.exists('/usr/bin/chromedriver')}")

# Initialize ChromeDriver
service = Service(chrome_driver_path, log_path="chromedriver.log")
driver = None
try:
    driver = webdriver.Chrome(service=service, options=options)
    print("Selenium session created successfully")
    # Test proxy
    driver.get("https://api.ipify.org")
    ip = driver.page_source
    print(f"Your IP: {ip}")
    driver.get("https://whatismyipaddress.com")
    driver.save_screenshot("ip_check.png")
    print("Screenshot saved as ip_check.png")
except Exception as e:
    print(f"Error: {e}")
finally:
    if driver:
        driver.quit()

Chrome binary exists: True
ChromeDriver exists: True
Error: Message: session not created
from unknown error: no chrome binary at /opt/google/chrome/google-chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
#0 0x628a5ab1a95a <unknown>
#1 0x628a5a5bb536 <unknown>
#2 0x628a5a5f2ea3 <unknown>
#3 0x628a5a5f1cdf <unknown>
#4 0x628a5a640fa9 <unknown>
#5 0x628a5a6406b6 <unknown>
#6 0x628a5a6324f3 <unknown>
#7 0x628a5a5fe9c8 <unknown>
#8 0x628a5a5ff661 <unknown>
#9 0x628a5aadf8df <unknown>
#10 0x628a5aae3a28 <unknown>
#11 0x628a5aac699c <unknown>
#12 0x628a5aae41ee <unknown>
#13 0x628a5aaae52e <unknown>
#14 0x628a5ab07be8 <unknown>
#15 0x628a5ab07dff <unknown>
#16 0x628a5ab1944c <unknown>
#17 0x7b184f494ac3 <unknown>



In [100]:
!which google-chrome

/usr/bin/google-chrome


SyntaxError: invalid syntax (410401063.py, line 1)